In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

In [2]:
df = pd.read_csv('data/hmda_lar.csv');

/home/victor/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (30,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print('Data shape: ', df.shape)

Data shape:  (17147, 47)


In [4]:
df.head()

,tract_to_msamd_income,rate_spread,population,minority_population,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_amount_000s,hud_median_family_income,applicant_income_000s,state_name,...,applicant_sex_name,applicant_race_name_5,applicant_race_name_4,applicant_race_name_3,applicant_race_name_2,applicant_race_name_1,applicant_ethnicity_name,agency_name,agency_abbr,action_taken_name
0,113.360001,NaN,4231,66.510002,899,1628,544,73700,96.0,New York,...,Male,NaN,NaN,NaN,NaN,White,Hispanic or Latino,Department of Housing and Urban Development,HUD,Loan originated
1,109.029999,NaN,7167,81.080002,1152,2008,320,73700,72.0,New York,...,Female,NaN,NaN,NaN,NaN,Asian,Not Hispanic or Latino,Department of Housing and Urban Development,HUD,Loan originated
2,139.039993,NaN,3199,34.040001,795,1252,314,73700,235.0,New York,...,Male,NaN,NaN,NaN,NaN,"Information not provided by applicant in mail,...","Information not provided by applicant in mail,...",Department of Housing and Urban Development,HUD,Loan originated
3,217.190002,NaN,6403,38.509998,516,16,548,73700,170.0,New York,...,Male,NaN,NaN,NaN,NaN,White,Not Hispanic or Latino,Consumer Financial Protection Bureau,CFPB,Loan originated
4,109.029999,NaN,7167,81.080002,1152,2008,343,73700,82.0,New York,...,Male,NaN,NaN,NaN,NaN,White,Hispanic or Latino,Department of Housing and Urban Development,HUD,Loan originated


In [5]:
samples = [df[col].dropna().sample().values[0] if df[col].dropna().shape[0] > 0 else np.nan for col in df.columns]

In [6]:
def generate_summary(df):
    samples = [df[col].dropna().sample().values[0] if df[col].dropna().shape[0] > 0 else np.nan for col in df.columns]
    return pd.DataFrame({'NA Count' : df.isna().sum(), 
                  'Unique Count' : df.nunique(),
                  'Type' : df.dtypes,
                  'Sample' : samples})

In [9]:
df_summary = generate_summary(df)
df_summary

,NA Count,Unique Count,Type,Sample
tract_to_msamd_income,0,618,float64,61.01
rate_spread,16251,221,float64,2.42
population,0,610,int64,7318
minority_population,0,579,float64,48.07
number_of_owner_occupied_units,0,474,int64,315
number_of_1_to_4_family_units,0,517,int64,693
loan_amount_000s,0,1126,int64,165
hud_median_family_income,0,1,int64,73700
applicant_income_000s,1453,592,float64,106
state_name,0,1,object,New York


In [10]:
cols_to_del = df_summary.index.values[df_summary.loc[:,'NA Count'] == df.shape[0]]
cols_to_del = set(cols_to_del)
cols_to_del

{'applicant_race_name_4',
 'applicant_race_name_5',
 'application_date_indicator',
 'co_applicant_race_name_4',
 'co_applicant_race_name_5',
 'denial_reason_name_1',
 'denial_reason_name_2',
 'denial_reason_name_3',
 'edit_status_name',
 'sequence_number'}

In [11]:
cols_to_del = cols_to_del.union(set(['applicant_race_name_' + str(i) for i in range(1,6)]))
cols_to_del = cols_to_del.union(set(['co_applicant_race_name_' + str(i) for i in range(2,6)]))
cols_to_del = cols_to_del.union(set(['denial_reason_name_' + str(i) for i in range(1,4)]))

In [12]:
df.drop(list(cols_to_del), axis=1, inplace=True)

In [13]:
df['co_applicant_sex_name'].unique()

array(['Female', 'Male', 'No co-applicant',
       'Information not provided by applicant in mail, Internet, or telephone application',
       'Not applicable'], dtype=object)

In [14]:
generate_summary(df)

,NA Count,Unique Count,Type,Sample
tract_to_msamd_income,0,618,float64,109.38
rate_spread,16251,221,float64,1.63
population,0,610,int64,4978
minority_population,0,579,float64,29.74
number_of_owner_occupied_units,0,474,int64,314
number_of_1_to_4_family_units,0,517,int64,1279
loan_amount_000s,0,1126,int64,550
hud_median_family_income,0,1,int64,73700
applicant_income_000s,1453,592,float64,56
state_name,0,1,object,New York


In [15]:
df.to_csv('data/cleaned_mortgage.csv')

In [7]:
st = StanfordNERTagger('models/stanford-ner/classifiers/english.muc.7class.distsim.crf.ser.gz',
                       'models/stanford-ner/stanford-ner.jar') 

In [8]:
for col, text in zip(df_summary.index, samples):
    tokenized_text = word_tokenize(str(text))
    classified_text = st.tag(tokenized_text)
    print('Column: ', col, '\n', classified_text, '\n')

Column:  tract_to_msamd_income 
 [('135.94999694824222', 'O')] 

Column:  rate_spread 
 [('3.0', 'O')] 

Column:  population 
 [('2112', 'O')] 

Column:  minority_population 
 [('50.38999938964844', 'O')] 

Column:  number_of_owner_occupied_units 
 [('138', 'O')] 

Column:  number_of_1_to_4_family_units 
 [('956', 'O')] 

Column:  loan_amount_000s 
 [('326', 'O')] 

Column:  hud_median_family_income 
 [('73700', 'O')] 

Column:  applicant_income_000s 
 [('126.0', 'O')] 

Column:  state_name 
 [('New', 'LOCATION'), ('York', 'LOCATION')] 

Column:  state_abbr 
 [('NY', 'O')] 

Column:  sequence_number 
 [('nan', 'O')] 

Column:  respondent_id 
 [('0000476810', 'O')] 

Column:  purchaser_type_name 
 [('Loan', 'O'), ('was', 'O'), ('not', 'O'), ('originated', 'O'), ('or', 'O'), ('was', 'O'), ('not', 'O'), ('sold', 'O'), ('in', 'O'), ('calendar', 'O'), ('year', 'O'), ('covered', 'O'), ('by', 'O'), ('register', 'O')] 

Column:  property_type_name 
 [('One-to-four', 'O'), ('family', 'O'), ('dw

In [9]:
df['applicant_race_name_1'].unique()

array(['White', 'Asian',
       'Information not provided by applicant in mail, Internet, or telephone application',
       'Black or African American', 'Not applicable',
       'American Indian or Alaska Native',
       'Native Hawaiian or Other Pacific Islander'], dtype=object)

In [10]:
df['applicant_sex_name'].unique()

array(['Male', 'Female', 'Not applicable',
       'Information not provided by applicant in mail, Internet, or telephone application'],
      dtype=object)

In [11]:
df['loan_purpose_name'].unique()

array(['Refinancing', 'Home purchase', 'Home improvement'], dtype=object)

In [12]:
nlp = en_core_web_sm.load()

In [13]:
# Annotations can be found here: https://spacy.io/api/annotation
for col, text in zip(df_summary.index, samples):
    doc = nlp(str(text))
    print('\n', 'Column: ', col)
    for ent in doc.ents:
        print('Text: ', ent.text, ';\tLabel: ', ent.label_)


 Column:  tract_to_msamd_income

 Column:  rate_spread
Text:  3.0 ;	Label:  CARDINAL

 Column:  population
Text:  2112 ;	Label:  CARDINAL

 Column:  minority_population
Text:  50.38999938964844 ;	Label:  CARDINAL

 Column:  number_of_owner_occupied_units
Text:  138 ;	Label:  CARDINAL

 Column:  number_of_1_to_4_family_units
Text:  956 ;	Label:  CARDINAL

 Column:  loan_amount_000s
Text:  326 ;	Label:  CARDINAL

 Column:  hud_median_family_income
Text:  73700 ;	Label:  CARDINAL

 Column:  applicant_income_000s
Text:  126.0 ;	Label:  CARDINAL

 Column:  state_name
Text:  New York ;	Label:  GPE

 Column:  state_abbr

 Column:  sequence_number

 Column:  respondent_id
Text:  0000476810 ;	Label:  CARDINAL

 Column:  purchaser_type_name
Text:  calendar year ;	Label:  DATE

 Column:  property_type_name
Text:  One ;	Label:  CARDINAL

 Column:  preapproval_name

 Column:  owner_occupancy_name

 Column:  msamd_name
Text:  New York ;	Label:  GPE
Text:  Jersey City ;	Label:  GPE
Text:  White Plai